In [1]:
#
# This file provides source code of XOR experiment using on NEAT-Python library
#
import os,signal,sys
import shutil

import neat

import math
import numpy as np



import multiprocessing
from CheckpointerBest import CheckpointerBest

In [2]:
# The current working directory
local_dir = "./"
out_dir = os.path.join(local_dir, 'checkpoints')
os.makedirs(out_dir, exist_ok=True)

In [82]:
def eval_fitness(net):
  
    global gens
    
    error_sum = 0.0
    outputs = []
    accs = []

    def _imp():

        fitness = 0
        for xi, xo in zip(xx, yy):        
            output = net.activate([xi]) 
            xacc =  1-abs(xo-output)
            fitness += xacc

        fitness = np.mean((fitness/len(xx)))

        return fitness    
    
    
    fitness = (_imp())*100
    fitness =np.round(fitness,decimals=4)
    fitness=max(fitness,-1000.)
    
    
    return fitness


In [83]:
from pytorch_neat.recurrent_net import RecurrentNet

    
def eval_genomes_mp(genomes, config):


    net = neat.nn.FeedForwardNetwork.create(genomes, config)
    genomes.fitness = eval_fitness(net)
    return genomes.fitness

def eval_genomes_single(genomes, config):
    #single process
    for genome_id, genome in genomes:
        #net = RecurrentNet.create(genome, config,1)
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = eval_fitness(net)
        

def createPoolAndConfig(config_file,checkpoint):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation,
                         config_file)
    

    
    # Create the population, which is the top-level object for a NEAT run.
    
    if checkpoint is not None:
        p = neat.Checkpointer.restore_checkpoint(checkpoint)
    else:
        p = neat.Population(config)
    
    return p, config

    
def run_experiment(config_file, checkpoint = None, mp = False):
    
    
    best_genome = None
    
    p,config=createPoolAndConfig(config_file,checkpoint)    
    
    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(False))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    p.add_reporter(CheckpointerBest( filename_prefix='checkpoints/sin_exp-checkpoint-'))
    pe = None
    
    
    #this part required to handle keyboard intterrupt correctly, and return population and config to evaluate test set.
    try:
       
        if mp:
            original_sigint_handler = signal.signal(signal.SIGINT, signal.SIG_IGN)

            pe = neat.ParallelEvaluator(multiprocessing.cpu_count(), eval_genomes_mp)
            
            signal.signal(signal.SIGINT, original_sigint_handler)

            """set_trace()
            return"""
            best_genome = p.run(pe.evaluate, 113551)    
        else:

            best_genome = p.run(eval_genomes_single, 1113551)
            #print('\nBest genome:\n{!s}'.format(best_genome))
    except:
        print("Stopping the Jobs. ", sys.exc_info())
        if mp:
            
            pe.pool.terminate()
            pe.pool.join()
            print("pool ok")
            
        return p,config        

    return p,config
    



In [84]:
def evaluate_best(p,best_genome,config):

    net = neat.nn.FeedForwardNetwork.create(best_genome, config)
    accs = []
    for xi, xo in zip(xx, yy):
        output = net.activate([xi])
        xacc =  1-(abs(xo-output))
        accs.append(xacc)

    print("\nmean acc {}\n".format(
            np.round(np.array(accs).mean()* 100.,decimals=2) 
    ))
    

In [85]:
#create full sin list 1 step degrees
degrees2radians = np.radians(np.arange(0,360,1))

#samples
sample_count = 45

xx = np.random.choice(degrees2radians,sample_count,replace=False)
yy = np.sin(xx)

print(len(degrees2radians), "samples: ", sample_count)

360 samples:  45


In [86]:
# Clean results of previous run if any or init the ouput directory
config_path = os.path.join(local_dir, 'sin_config.ini')

# Run the experiment __ To continue from a checkpoint use cp variable , eg cp=1024
cp = 1147
if cp is not None:
    ret = run_experiment(config_path, checkpoint='checkpoints/sin_exp-checkpoint-{}'.format(cp), mp=True)
else:
    ret = run_experiment(config_path, mp=True)




 ****** Running generation 1147 ****** 

Population's average fitness: 27.82006 stdev: 92.75112
Best fitness: 80.62010 - size: (20, 13) - species 98 - id 37365
Saving checkpoint to checkpoints/sin_exp-checkpoint-1147
Average adjusted fitness: 0.949
Mean genetic distance 2.446, standard deviation 0.680
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.406 sec

 ****** Running generation 1148 ****** 

Population's average fitness: 8.74091 stdev: 162.00641
Best fitness: 80.62010 - size: (20, 13) - species 98 - id 37365
Average adjusted fitness: 0.934
Mean genetic distance 2.371, standard deviation 0.647
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.116 sec (0.261 average)

 ****** Running generation 1149 ****** 

Population's average fitness: 29.23962 stdev: 96.90612
Best fitness: 80.62010 - size: (20, 13) - species 98 - id 37365
Average adjusted fitness: 0.953
Mean genetic distance 2.330, standard deviation 0.714
Population o

Population's average fitness: 6.45335 stdev: 152.05972
Best fitness: 80.62010 - size: (20, 13) - species 98 - id 37365
Average adjusted fitness: 0.932
Mean genetic distance 2.557, standard deviation 0.751
Population of 126 members in 3 species
Total extinctions: 0
Generation time: 0.121 sec (0.127 average)

 ****** Running generation 1173 ****** 

Population's average fitness: 5.41294 stdev: 165.91083
Best fitness: 80.62010 - size: (20, 13) - species 98 - id 37365
Average adjusted fitness: 0.930
Mean genetic distance 2.561, standard deviation 0.707
Population of 124 members in 3 species
Total extinctions: 0
Generation time: 0.116 sec (0.126 average)

 ****** Running generation 1174 ****** 

Population's average fitness: 5.04212 stdev: 171.44566
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Saving checkpoint to checkpoints/sin_exp-checkpoint-1174
Average adjusted fitness: 0.930
Mean genetic distance 2.571, standard deviation 0.733
Population of 126 members in 3 species
T

Population's average fitness: 34.21566 stdev: 29.06194
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.806
Mean genetic distance 2.601, standard deviation 0.700
Population of 125 members in 3 species
Total extinctions: 0
Generation time: 0.120 sec (0.126 average)

 ****** Running generation 1197 ****** 

Population's average fitness: 39.68388 stdev: 34.15495
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.862
Mean genetic distance 2.594, standard deviation 0.703
Population of 124 members in 3 species
Total extinctions: 0
Generation time: 0.118 sec (0.126 average)

 ****** Running generation 1198 ****** 

Population's average fitness: 26.39030 stdev: 101.69413
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.934
Mean genetic distance 2.480, standard deviation 0.691
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.122 sec (0.125 averag

Population's average fitness: 8.61113 stdev: 166.33255
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.932
Mean genetic distance 2.552, standard deviation 0.572
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.121 sec (0.117 average)

 ****** Running generation 1221 ****** 

Population's average fitness: 17.49189 stdev: 138.76333
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.942
Mean genetic distance 2.403, standard deviation 0.683
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.120 sec (0.117 average)

 ****** Running generation 1222 ****** 

Population's average fitness: 7.95890 stdev: 169.16012
Best fitness: 80.62190 - size: (9, 6) - species 131 - id 3194
Average adjusted fitness: 0.933
Mean genetic distance 2.455, standard deviation 0.641
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.125 sec (0.118 averag

Population's average fitness: -1.59977 stdev: 181.58664
Best fitness: 80.62190 - size: (8, 3) - species 131 - id 9072
Average adjusted fitness: 0.924
Mean genetic distance 2.504, standard deviation 0.675
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.125 sec (0.118 average)

 ****** Running generation 1245 ****** 

Population's average fitness: 19.82988 stdev: 134.63411
Best fitness: 80.62190 - size: (8, 3) - species 131 - id 9072
Average adjusted fitness: 0.944
Mean genetic distance 2.448, standard deviation 0.674
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.122 sec (0.118 average)

 ****** Running generation 1246 ****** 

Population's average fitness: 27.52162 stdev: 100.42571
Best fitness: 80.62190 - size: (8, 3) - species 131 - id 9072
Average adjusted fitness: 0.951
Mean genetic distance 2.498, standard deviation 0.637
Population of 125 members in 2 species
Total extinctions: 0
Generation time: 0.122 sec (0.119 aver

...

Population's average fitness: -3.76057 stdev: 187.93717
Best fitness: 80.62650 - size: (7, 3) - species 151 - id 56570
Average adjusted fitness: 0.924
Mean genetic distance 2.746, standard deviation 0.760
Population of 124 members in 5 species
Total extinctions: 0
Generation time: 0.106 sec (0.110 average)

 ****** Running generation 1704 ****** 

Population's average fitness: 36.99239 stdev: 44.03974
Best fitness: 80.62650 - size: (7, 3) - species 151 - id 56570
Average adjusted fitness: 0.915
Mean genetic distance 2.785, standard deviation 0.738
Population of 127 members in 6 species
Total extinctions: 0
Generation time: 0.102 sec (0.109 average)

 ****** Running generation 1705 ****** 

Population's average fitness: 34.62322 stdev: 60.35960
Best fitness: 80.62650 - size: (7, 3) - species 151 - id 56570
Average adjusted fitness: 0.933
Mean genetic distance 2.767, standard deviation 0.746
Population of 125 members in 6 species
Total extinctions: 0
Generation time: 0.110 sec (0.109 ave

Population's average fitness: 34.99304 stdev: 53.59486
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.919
Mean genetic distance 2.999, standard deviation 0.796
Population of 124 members in 7 species
Total extinctions: 0
Generation time: 0.108 sec (0.107 average)

 ****** Running generation 1729 ****** 

Population's average fitness: 39.18157 stdev: 26.28562
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.845
Mean genetic distance 2.994, standard deviation 0.782
Population of 124 members in 8 species
Total extinctions: 0
Generation time: 0.108 sec (0.107 average)

 ****** Running generation 1730 ****** 

Population's average fitness: 12.10687 stdev: 160.63365
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.942
Mean genetic distance 3.005, standard deviation 0.772
Population of 124 members in 8 species
Total extinctions: 0
Generation time: 0.111 sec (0.108 ave

Population's average fitness: 34.64202 stdev: 64.39506
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.937
Mean genetic distance 2.564, standard deviation 0.772
Population of 127 members in 5 species
Total extinctions: 0
Generation time: 0.113 sec (0.114 average)

 ****** Running generation 1753 ****** 

Population's average fitness: 14.43716 stdev: 145.96313
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.939
Mean genetic distance 2.513, standard deviation 0.756
Population of 124 members in 5 species
Total extinctions: 0
Generation time: 0.112 sec (0.114 average)

 ****** Running generation 1754 ****** 

Population's average fitness: 5.96817 stdev: 128.08813
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.902
Mean genetic distance 2.493, standard deviation 0.716
Population of 123 members in 5 species
Total extinctions: 0
Generation time: 0.118 sec (0.114 ave

Population's average fitness: 13.60215 stdev: 142.07775
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.939
Mean genetic distance 2.639, standard deviation 0.648
Population of 127 members in 6 species
Total extinctions: 0
Generation time: 0.115 sec (0.112 average)

 ****** Running generation 1777 ****** 

Population's average fitness: 16.55879 stdev: 138.94896
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.941
Mean genetic distance 2.526, standard deviation 0.662
Population of 123 members in 6 species
Total extinctions: 0
Generation time: 0.116 sec (0.113 average)

 ****** Running generation 1778 ****** 

Population's average fitness: 27.09534 stdev: 87.26433
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.918
Mean genetic distance 2.542, standard deviation 0.664
Population of 123 members in 6 species
Total extinctions: 0
Generation time: 0.111 sec (0.113 av

Population's average fitness: 3.83123 stdev: 181.89423
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.930
Mean genetic distance 2.808, standard deviation 0.733
Population of 126 members in 5 species
Total extinctions: 0
Generation time: 0.116 sec (0.113 average)

 ****** Running generation 1801 ****** 

Population's average fitness: 12.67292 stdev: 136.05186
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.938
Mean genetic distance 2.897, standard deviation 0.781
Population of 126 members in 5 species
Total extinctions: 0
Generation time: 0.116 sec (0.113 average)

 ****** Running generation 1802 ****** 

Population's average fitness: 16.70908 stdev: 137.00763
Best fitness: 80.62710 - size: (8, 4) - species 135 - id 65531
Average adjusted fitness: 0.942
Mean genetic distance 2.811, standard deviation 0.798
Population of 125 members in 5 species
Total extinctions: 0
Generation time: 0.116 sec (0.114 av

In [87]:
#if ret is None:
p, config = ret


In [88]:
evaluate_best(p,p.best_genome,config)


mean acc 80.63



In [89]:
#use full angles.
print(len(degrees2radians))

xx = degrees2radians
yy = np.sin(xx)
evaluate_best(p,p.best_genome,config)

360

mean acc 80.34



In [90]:
# use 0.01 degrees per step 

degrees2radians_d = np.radians(np.arange(0,360,.01))
print(len(degrees2radians_d))

xx = degrees2radians_d
yy = np.sin(xx)

evaluate_best(p,p.best_genome,config)

36000

mean acc 80.34



In [91]:
print(p.best_genome)

Key: 65531
Fitness: 80.6271
Nodes:
	0 DefaultNodeGene(key=0, bias=2.8750381327203356, response=1.0, activation=clamped, aggregation=min)
	181 DefaultNodeGene(key=181, bias=-1.265262350609702, response=1.0, activation=sigmoid, aggregation=min)
	189 DefaultNodeGene(key=189, bias=-2.6066015511397977, response=1.0, activation=exp, aggregation=median)
	191 DefaultNodeGene(key=191, bias=0.4154937260503494, response=1.0, activation=sigmoid, aggregation=max)
	192 DefaultNodeGene(key=192, bias=-0.13779391441370387, response=1.0, activation=log, aggregation=median)
	2546 DefaultNodeGene(key=2546, bias=-0.9702818533194371, response=1.0, activation=relu, aggregation=max)
	11759 DefaultNodeGene(key=11759, bias=1.6244235639814273, response=1.0, activation=clamped, aggregation=max)
	17645 DefaultNodeGene(key=17645, bias=0.9018595550639844, response=1.0, activation=relu, aggregation=min)
Connections:
	DefaultConnectionGene(key=(-1, 0), weight=-0.9164397233185276, enabled=True)
	DefaultConnectionGene(k